In [1]:
import sys
sys.path.append('../')

import cqt
import cqt.datagen as dg
import cqt.dbutility.dbutility as db
import cqt.model.asset_model as md
import cqt.model.asset_model_component_spot as comp_spot
import pandas as pd
import numpy as np

from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
from matplotlib import pyplot

from statsmodels.tsa import arima_model as arma
from sklearn.metrics import mean_squared_error

from sqlalchemy import create_engine

D:\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [6]:
key="59103194-7503-441E-A849-1B961471734B"
source = 'coinapi'
request_eth_20170901 = {'request_type' : 'ohlcv', \
               'symbol_id' : 'COINBASE_SPOT_ETH_USD', \
               'period_id' : '1DAY', \
               'time_start' : '2017-01-01T00:00:00', \
               'time_end' : '2017-09-01T00:00:00', \
               'limit' : '100000'}

#lala=dg.get_req_str(source, request_btc)
#print(lala)
df_eth_20170901= dg.data_gen(source, request_eth_20170901,key,write_to_file=False, returnDF=True)
df_eth_20170901


https://rest.coinapi.io/v1/ohlcv/COINBASE_SPOT_ETH_USD/history?period_id=1DAY&time_start=2017-01-01T00:00:00&time_end=2017-09-01T00:00:00&limit=100000


,price_close,price_open,price_high,price_low,time_close,time_open,trades_count,volume_traded,key
0,8.18,8.16,8.60,8.08,2017-01-01T23:58:25.0520000Z,2017-01-01T00:00:01.4630000Z,5815,59049.821264,2017-01-01
1,8.38,8.20,8.53,8.05,2017-01-02T23:58:31.2030000Z,2017-01-02T00:00:00.9760000Z,8632,70376.514494,2017-01-02
2,9.59,8.38,10.02,8.24,2017-01-03T23:59:08.6690000Z,2017-01-03T00:00:01.4100000Z,16972,190986.091366,2017-01-03
3,11.17,9.62,11.17,9.34,2017-01-04T23:59:31.5150000Z,2017-01-04T00:00:01.6060000Z,23380,248946.575232,2017-01-04
4,10.23,11.13,11.80,9.08,2017-01-05T23:59:30.9480000Z,2017-01-05T00:00:00.8060000Z,54460,311007.409866,2017-01-05
5,10.14,10.23,10.46,9.52,2017-01-06T23:59:31.4930000Z,2017-01-06T00:00:05.9030000Z,13342,137058.389702,2017-01-06
6,9.92,10.14,10.21,9.55,2017-01-07T23:59:01.3580000Z,2017-01-07T00:00:01.6290000Z,8260,66797.597883,2017-01-07
7,10.50,9.92,10.58,9.89,2017-01-08T23:59:31.0450000Z,2017-01-08T00:00:31.3680000Z,8336,62164.149788,2017-01-08
8,10.45,10.43,10.45,10.43,2017-01-09T07:18:01.4060000Z,2017-01-09T07:18:01.4060000Z,6,173.149730,2017-01-09
9,10.65,10.44,10.72,10.44,2017-01-10T22:03:31.2230000Z,2017-01-10T04:47:53.4880000Z,57,373.816155,2017-01-10


In [7]:
tbl_name = dg.get_req_str(source, request_eth_20170901,False)
print(tbl_name)

coinapi-ohlcv-COINBASE_SPOT_ETH_USD-1DAY


In [9]:

db.dump_to_db(df_eth_20170901,tbl_name,'key')

In [10]:
key="59103194-7503-441E-A849-1B961471734B"
source = 'coinapi'
request_eth_now = {'request_type' : 'ohlcv', \
               'symbol_id' : 'COINBASE_SPOT_ETH_USD', \
               'period_id' : '1DAY', \
               'time_start' : '2017-08-01T00:00:00', \
               'limit' : '100000'}

#lala=dg.get_req_str(source, request_btc)
#print(lala)
df_eth_now= dg.data_gen(source, request_eth_now,key,write_to_file=False, returnDF=True)
df_eth_now


https://rest.coinapi.io/v1/ohlcv/COINBASE_SPOT_ETH_USD/history?period_id=1DAY&time_start=2017-08-01T00:00:00&limit=100000


,price_close,price_open,price_high,price_low,time_close,time_open,trades_count,volume_traded,key
0,225.73,200.83,232.73,200.82,2017-08-01T23:59:50.4490000Z,2017-08-01T00:00:03.5260000Z,75648,435730.345923,2017-08-01
1,217.92,225.73,228.80,215.37,2017-08-02T23:59:58.8630000Z,2017-08-02T00:00:03.7850000Z,35120,149917.377440,2017-08-02
2,224.00,217.92,227.53,217.35,2017-08-03T23:59:57.0450000Z,2017-08-03T00:00:03.8770000Z,29741,113161.874522,2017-08-03
3,220.72,224.00,226.82,219.51,2017-08-04T23:59:53.3040000Z,2017-08-04T00:00:03.5930000Z,28673,98578.273710,2017-08-04
4,251.85,220.72,258.98,219.54,2017-08-05T23:59:47.8310000Z,2017-08-05T00:00:03.4860000Z,65156,319645.767786,2017-08-05
5,263.96,251.86,273.00,250.99,2017-08-06T23:59:52.4320000Z,2017-08-06T00:00:01.8970000Z,55356,238836.553953,2017-08-06
6,269.90,263.97,274.81,257.10,2017-08-07T23:59:58.5260000Z,2017-08-07T00:00:02.4490000Z,44362,168589.607547,2017-08-07
7,295.93,269.89,299.45,267.00,2017-08-08T23:59:50.7710000Z,2017-08-08T00:00:01.2170000Z,56743,267302.649304,2017-08-08
8,295.26,295.95,314.71,276.55,2017-08-09T23:59:51.3550000Z,2017-08-09T00:00:02.1480000Z,88888,465341.003687,2017-08-09
9,300.51,295.26,309.80,288.34,2017-08-10T23:59:58.5540000Z,2017-08-10T00:00:03.2180000Z,40462,154225.487929,2017-08-10


In [11]:
tbl_name='coinapi-ohlcv-COINBASE_SPOT_ETH_USD-1DAY'
db.dump_to_db(df_eth_now,tbl_name,'key')

In [12]:
tbl_name='coinapi-ohlcv-COINBASE_SPOT_ETH_USD-1DAY'
laladf=db.get_from_db(tbl_name,from_date='2017-11-30',to_date='2018/04/16')

In [14]:
laladf.data

,price_close,price_open,price_high,price_low,time_close,time_open,trades_count,volume_traded,key,LastUpdated
0,434.00,434.01,470.00,401.50,2017-11-30T23:59:59.5550000Z,2017-11-30T00:00:00.7970000Z,186864,362765.245598,2017-11-30,2018-06-19 22:42:04.087846
1,463.82,434.00,468.43,421.57,2017-12-01T23:59:58.5070000Z,2017-12-01T00:00:00.4150000Z,127714,195010.419417,2017-12-01,2018-06-19 22:42:04.087846
2,460.34,463.82,474.96,451.21,2017-12-02T23:59:54.9710000Z,2017-12-02T00:00:00.0660000Z,240849,162582.069582,2017-12-02,2018-06-19 22:42:04.087846
3,465.00,460.35,483.61,450.00,2017-12-03T23:59:59.9990000Z,2017-12-03T00:00:00.1110000Z,236053,167096.251216,2017-12-03,2018-06-19 22:42:04.087846
4,468.22,465.00,473.00,449.40,2017-12-04T23:59:51.7190000Z,2017-12-04T00:00:00.1450000Z,137678,135389.881073,2017-12-04,2018-06-19 22:42:04.087846
5,457.30,468.22,470.00,452.00,2017-12-05T23:59:47.9810000Z,2017-12-05T00:00:00.4350000Z,144990,153047.682357,2017-12-05,2018-06-19 22:42:04.087846
6,432.49,457.29,461.99,425.01,2017-12-06T23:59:57.9520000Z,2017-12-06T00:00:00.3550000Z,230313,349323.666095,2017-12-06,2018-06-19 22:42:04.087846
7,433.27,432.48,448.89,415.01,2017-12-07T23:59:54.0740000Z,2017-12-07T00:00:00.6010000Z,177856,330768.805504,2017-12-07,2018-06-19 22:42:04.087846
8,463.45,432.82,479.10,420.01,2017-12-08T23:59:55.7840000Z,2017-12-08T00:00:03.5360000Z,183130,395454.082134,2017-12-08,2018-06-19 22:42:04.087846
9,485.04,463.45,535.10,450.00,2017-12-09T23:59:58.8290000Z,2017-12-09T00:00:02.2140000Z,370993,544362.402031,2017-12-09,2018-06-19 22:42:04.087846
